**Import necessary libraries**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import calendar
import matplotlib.pyplot as plt
# import seaborn as sns
%matplotlib inline

**Load and preprocess the dataset**

In [2]:
data = pd.read_csv('Transaksi.csv', delimiter=';')
print(data.head())

                           id_transaksi    nama        kategori id_jenis  \
0  01671a35-fe22-4b23-bc97-e9cb7069cb25    Adam  Transportation   keluar   
1  01fef77f-4632-4e71-bdd8-8282cf8d03ab    Gaby           Other   keluar   
2  021afb36-b69c-4e81-8c3a-71957d74856e  Galang          Family   keluar   
3  03818dcb-c12c-4837-9768-faf97584b2b9   Fajri  Transportation   keluar   
4  07167e71-4b92-41b7-8533-cf94a24a9f32   Andra            Food   keluar   

      aset           tanggal  jumlah note      UMR  \
0     cash  27/10/2023 00.00   10000    -  2943033   
1      BNI  12/12/2023 00.00   83000    -  1986670   
2  mandiri  05/09/2023 00.00  398000    -  1981782   
3      BNI  08/09/2023 00.00   15000    -  2661280   
4     cash  24/11/2023 00.00  216000    -  4900000   

                       daerah        created_at        updated_at  
0                       Jambi  14/12/2023 14.38  14/12/2023 14.38  
1                  Jawa Barat  11/12/2023 05.23  11/12/2023 05.23  
2  Daerah Isti

**Feature selection**

In [3]:
# Customize this based on your actual dataset columns. Select relevant features for clustering.
features = data[['nama', 'kategori', 'id_jenis', 'aset', 'tanggal', 'jumlah', 'UMR', 'daerah']]
print(data.columns)
features.head()

Index(['id_transaksi', 'nama', 'kategori', 'id_jenis', 'aset', 'tanggal',
       'jumlah', 'note', 'UMR', 'daerah', 'created_at', 'updated_at'],
      dtype='object')


,nama,kategori,id_jenis,aset,tanggal,jumlah,UMR,daerah
0,Adam,Transportation,keluar,cash,27/10/2023 00.00,10000,2943033,Jambi
1,Gaby,Other,keluar,BNI,12/12/2023 00.00,83000,1986670,Jawa Barat
2,Galang,Family,keluar,mandiri,05/09/2023 00.00,398000,1981782,Daerah Istimewa Yogyakarta
3,Fajri,Transportation,keluar,BNI,08/09/2023 00.00,15000,2661280,Banten
4,Andra,Food,keluar,cash,24/11/2023 00.00,216000,4900000,DKI Jakarta


**Standardize the features**

In [4]:
scaler = OneHotEncoder()
scaled_features = scaler.fit_transform(features)
scaled_features_dense = csr_matrix.toarray(scaled_features)

**Implementing K-means clustering**

In [5]:
kmeans = KMeans(n_clusters=3, random_state=42)
data['cluster'] = kmeans.fit_predict(scaled_features)

# Convert the 'tanggal' column to datetime format
data['tanggal'] = pd.to_datetime(data['tanggal'], format='%d/%m/%Y %H.%M')

# Create a new column for the month
data['bulan'] = data['tanggal'].dt.month

# Map the month numbers to month names in Indonesian
data['bulan'] = data['bulan'].apply(lambda x: calendar.month_name[x])

# Group the data by 'nama', 'bulan', and 'id_jenis' and calculate the total amount
total_transactions = data.groupby(['nama', 'bulan', 'id_jenis'])['jumlah'].sum()

# Print the results
print(total_transactions)


nama   bulan      id_jenis
Abdul  November   keluar       2388000
       October    keluar        467000
                  masuk         160000
       September  keluar       1642000
Adam   October    keluar      48152000
                                ...   
Ziva   November   keluar        237000
       October    keluar        622000
       September  keluar         15000
Zulfa  December   keluar      25641900
       November   masuk       68600000
Name: jumlah, Length: 1163, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [6]:
# Label kluster untuk setiap data
labels = kmeans.labels_

# Menambahkan kolom 'Cluster' ke dataframe data
data['Cluster'] = labels

# Menentukan klasifikasi pengeluaran berdasarkan UMR
data.loc[data['jumlah'] > 0.8 * data['UMR'], 'Pengeluaran'] = 'Boros'
data.loc[(data['jumlah'] >= 0.5 * data['UMR']) & (data['jumlah'] <= 0.8 * data['UMR']), 'Pengeluaran'] = 'Hemat'
data.loc[data['jumlah'] < 0.5 * data['UMR'], 'Pengeluaran'] = 'Biasa Saja'

# Mengklasifikasikan pemasukan berdasarkan UMR
data['Kategori_Pemasukan'] = ''
data.loc[data['jumlah'] > 0.8 * data['UMR'], 'Kategori_Pemasukan'] = 'Boros'
data.loc[(data['jumlah'] >= 0.5 * data['UMR']) & (data['jumlah'] <= 0.8 * data['UMR']), 'Kategori_Pemasukan'] = 'Hemat'
data.loc[data['jumlah'] < 0.5 * data['UMR'], 'Kategori_Pemasukan'] = 'Biasa Saja'

data['Klasifikasi'] = ''

# Klasifikasi berdasarkan kombinasi kategori pengeluaran dan pemasukan
data.loc[(data['Pengeluaran'] == 'Boros') | (data['Kategori_Pemasukan'] == 'Biasa Saja'), 'Klasifikasi'] = 'Boros'
data.loc[(data['Pengeluaran'] == 'Hemat') | (data['Kategori_Pemasukan'] == 'Biasa Saja'), 'Klasifikasi'] = 'Hemat'
data.loc[(data['Pengeluaran'] == 'Biasa Saja') | (data['Kategori_Pemasukan'] == 'Biasa Saja'), 'Klasifikasi'] = 'Biasa Saja'

# Menampilkan hasil klasifikasi dan kesimpulan per pengguna
for user in data['nama'].unique():
    user_data = data[data['nama'] == user]
    klasifikasi = user_data['Klasifikasi'].unique()[0]
    print(f"Nama: {user}")
    print(f"Klasifikasi: {klasifikasi}")
    print()

Nama: Adam
Klasifikasi: Biasa Saja

Nama: Gaby
Klasifikasi: Biasa Saja

Nama: Galang
Klasifikasi: Biasa Saja

Nama: Fajri
Klasifikasi: Biasa Saja

Nama: Andra
Klasifikasi: Biasa Saja

Nama: Fahri
Klasifikasi: Boros

Nama: Zulfa
Klasifikasi: Biasa Saja

Nama: Yana
Klasifikasi: Biasa Saja

Nama: Amin
Klasifikasi: Biasa Saja

Nama: Liona
Klasifikasi: Biasa Saja

Nama: Mark
Klasifikasi: Biasa Saja

Nama: Yudit
Klasifikasi: Biasa Saja

Nama: Zahra
Klasifikasi: Biasa Saja

Nama: Fauziah
Klasifikasi: Biasa Saja

Nama: Salwa
Klasifikasi: Boros

Nama: Vania
Klasifikasi: Biasa Saja

Nama: Jamal
Klasifikasi: Boros

Nama: Jessica
Klasifikasi: Hemat

Nama: James
Klasifikasi: Biasa Saja

Nama: Stella
Klasifikasi: Biasa Saja

Nama: Lia
Klasifikasi: Biasa Saja

Nama: Ayu
Klasifikasi: Boros

Nama: Agnes
Klasifikasi: Biasa Saja

Nama: Ahmad
Klasifikasi: Biasa Saja

Nama: Kevin
Klasifikasi: Biasa Saja

Nama: Johnny
Klasifikasi: Biasa Saja

Nama: Dian
Klasifikasi: Biasa Saja

Nama: Nicholas
Klasifikasi: B

**Model Rekomendasi**

In [7]:
def recommendation_model(data, user):
    user_data = data[data['nama'] == user]
    klasifikasi = user_data['Klasifikasi'].unique()[0]

    if klasifikasi == 'Boros':
        recommendation = "Anda termasuk dalam kategori pengeluaran boros. Rekomendasi: Kurangi pengeluaran yang tidak perlu dan buat anggaran yang lebih ketat."
    elif klasifikasi == 'Hemat':
        recommendation = "Anda termasuk dalam kategori pengeluaran hemat. Rekomendasi: Lanjutkan kebiasaan hemat Anda dan pertimbangkan untuk menabung lebih banyak."
    else:
        recommendation = "Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk mengatur anggaran dengan lebih efisien."

    return recommendation

# Contoh penggunaan model rekomendasi
users = data['nama'].unique()
for user in users:
    recommendation = recommendation_model(data, user)
    print(f"Nama: {user}")
    print(f"Klasifikasi: {data[data['nama'] == user]['Klasifikasi'].unique()[0]}")
    print(f"Rekomendasi: {recommendation}")
    print()

Nama: Adam
Klasifikasi: Biasa Saja
Rekomendasi: Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk mengatur anggaran dengan lebih efisien.

Nama: Gaby
Klasifikasi: Biasa Saja
Rekomendasi: Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk mengatur anggaran dengan lebih efisien.

Nama: Galang
Klasifikasi: Biasa Saja
Rekomendasi: Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk mengatur anggaran dengan lebih efisien.

Nama: Fajri
Klasifikasi: Biasa Saja
Rekomendasi: Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk mengatur anggaran dengan lebih efisien.

Nama: Andra
Klasifikasi: Biasa Saja
Rekomendasi: Anda termasuk dalam kategori pengeluaran biasa saja. Rekomendasi: Perhatikan pengeluaran Anda dan pertimbangkan untuk me

**Training and Testing**

In [8]:
kmeans.cluster_centers_

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, data['cluster'], test_size=0.2, random_state=42)
# Split the data into training and validation sets
X_train, X_val, _, _ = train_test_split(
    scaled_features_dense,
    data['id_jenis'],
    test_size=0.2,
    random_state=42
)

**Neural Network Model Using TensorFlow**

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=3, activation='softmax')  # 3 clusters
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10
114/114 [==============================] - 4s 18ms/step - loss: 0.4148 - accuracy: 0.8746 - val_loss: 0.0780 - val_accuracy: 0.9901
Epoch 2/10
114/114 [==============================] - 1s 10ms/step - loss: 0.0337 - accuracy: 0.9978 - val_loss: 0.0218 - val_accuracy: 0.9956
Epoch 3/10
114/114 [==============================] - 1s 8ms/step - loss: 0.0102 - accuracy: 0.9997 - val_loss: 0.0123 - val_accuracy: 0.9956
Epoch 4/10
114/114 [==============================] - 1s 9ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 0.9967
Epoch 5/10
114/114 [==============================] - 1s 6ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 0.9967
Epoch 6/10
114/114 [==============================] - 1s 7ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 0.9967
Epoch 7/10
114/114 [==============================] - 1s 7ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0060 - val_accuracy: 0.9967
Epoc

**Save Model**

In [10]:
# Save the model to a file
joblib.dump(model, 'cfm_model.pkl')

['cfm_model.pkl']